In [1]:
sc

org.apache.spark.SparkContext = org.apache.spark.SparkContext@441e789e

# Exercise 1: Using CQL search for items with ”girls" in the title. (15 minutes)

### Resources: (10 minutes reading)
* [CQL solr query overview](https://docs.datastax.com/en/datastax_enterprise/4.8/datastax_enterprise/srch/srchCql.html)
* [CQL solr_query JSON spec](https://docs.datastax.com/en/datastax_enterprise/4.8/datastax_enterprise/srch/srchJSON.html)
* [Solr HTTP RESTful API](https://docs.datastax.com/en/datastax_enterprise/4.8/datastax_enterprise/srch/srchHttp.html)

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"title:girls"}';

### Follow-up questions:
* Are there other ways to write this query? Give some exmple.

    (HTTP RESTful API, CQL solr_query JSON format, CQL solr_query non-JSON format)

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = 'title:girls';

Another variant (RESTful API): [http://54.200.196.81:8983/solr/amazon.metadata/select?q=title%3Agirls&wt=json&indent=true](http://54.200.196.81:8983/solr/amazon.metadata/select?q=title%3Agirls&wt=json&indent=true)

### Note: This is a good time to introduce Solr Admin UI [http://54.200.196.81:8983/solr](http://54.200.196.81:8983/solr).

### Another follow-up question:
* CQL solr_query is way more flexible than pure CQL query, but is there a catch? Compared to regular CQL query, what is the downside of using CQL solr_query from developer's perspective?

### Answer:

Since all query conditions will have to fit in solr_query, you can no longer effectively use prepared statement as you normally would be able to with pure CQL query.

# Question 2: What is the difference between main query ("q") and filter query ("fq") query parameter? Is there advantage of using one over the other? What is the best way to specify "fq" parameter if there are multiple AND conditions?

### Answer:

* "fq" won't influence document score while "q" will
* however, "fq" could have a pretty major performance advantage over "q" for complex queries that could reuse part of the query conditions, because "Each filter query is normally generated and cached independently in the filterCache, leading to improved request throughput for filters that are likely to be reused"
* You can use multiple "fq" clauses if there are multiple AND conditions, so each filter condition can be independently cached and reused

# Exercise 3: Using CQL search for ”girls" in the title. What categories contain results? (5 minutes)


### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"title:girls", "facet":{"field":"categories"}}';

# Exercise 4: Some of the categories in the facet output above contain zero result, how to eliminate those zero result categories in the output?

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"title:girls", "facet":{"field":"categories", "mincount":1}}';

# Exercise 5: Using CQL search for ”girls" in the title, filtered by category books. (5 minutes)

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"title:girls", "fq":"categories:books"}' limit 100;

# Exercise 6: From the above answer, find out number of books in different price ranges, such as 0-2, 2-4, 4-6, 6-8, 8-10, 10-12 dollars (5 minutes)
### Hint: Use range facet, https://cwiki.apache.org/confluence/display/solr/Faceting#Faceting-RangeFaceting

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"title:girls", "fq":"categories:books", "facet":{"range":"price", "range.start":0.0, "range.end":12.0, "range.gap":2.0}, "useFieldCache":true}' limit 100;

# Exercise 7: Using CQL search for ”girls" in the title, sort by title. (10 minutes)

### Answer:

In [ ]:
select * from amazon.metadata where solr_query = '{"q":"title:girls", "sort":"title asc"}';

### Why using "sort":"title asc" doesn't work? How to fix it?
### Answer: (Attempt #2)

In [ ]:
select asin, categories, title from amazon.metadata where solr_query = '{"q":"title:girls", "sort":"title asc", "useFieldCache":true}' limit 100;

### Now sort in the reverse order.
### Answer:

In [ ]:
select asin, categories, title from amazon.metadata where solr_query = '{"q":"title:girls", "sort":"title desc", "useFieldCache":true}' limit 100;

### What happened? Why the 2nd output (reverse order) is not the exact opposite of the first output?

### Answer:
See [DSP-9604](https://datastax.jira.com/browse/DSP-9604)

### Class Discussion: How to make the order in the sort output deterministic?

### Answer:

* Change the schema to make the title as a non-tokenized text field, instead of tokenized text field.
* Change title to string field, which is not tokenized.
* Sort on a field that is meaningful for sorting, such as ```asin```.

# Exercise 8: CQL solr_query is not fully functionally equivalent to Solr HTTP API (10 minutes)
Read [DSP-4761](https://datastax.jira.com/browse/DSP-4761), and write down a list of Solr functionalties that are possible with Solr HTTP API but not yet in CQL solr_query (DSE-4.8 and DSE-5.0).

### Answer: